In [1]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
all_data = pd.read_csv("voice.csv")
label = all_data.pop('label')

all_data = all_data.values

label.replace(['male','female'], [1, 0], inplace = True)
label = label.values

train_data, test_data, train_labels, test_labels = train_test_split(all_data, label, test_size = 0.2)

train_data, test_data, train_labels, test_labels = np.array(train_data, dtype = 'float32'), np.array(test_data, dtype = 'float32'),np.array(train_labels, dtype = 'float32'),np.array(test_labels, dtype = 'float32')

print(len(train_data))
print(len(test_data))

2534
634


In [3]:
label_1 = np.ones(len(train_labels)/2)
label_2 = np.zeros(len(train_labels)/2)
labels = np.concatenate((label_1,label_2), axis = 0)

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


In [4]:
D1 = np.zeros((1,20))
D2 = np.zeros((1,20))
for i in range(0,len(train_labels)):
   if labels[i]==1:
       D1 = np.vstack((D1, train_data[i,:].reshape(1,20)))
   else:
       D2 = np.vstack((D2, train_data[i,:].reshape(1,20)))

D1 = np.delete(D1,0,0)
D2 = np.delete(D2,0,0)

In [5]:
sd1 = len(D1)
sd2 = len(D2)
mu1 = np.mean(D1, axis = 0)
mu2 = np.mean(D2, axis = 0)

dd1 = D1 - ((mu1) * np.ones((sd1,1)))
dd2 = D2 - ((mu2) * np.ones((sd2,1)))

In [6]:
cov1 = (np.dot(dd1.T, dd1))/sd1 + 0.0000001 * np.identity(20)
cov2 = (np.dot(dd2.T, dd2))/sd2 + 0.0000001 * np.identity(20)

icov1 = np.linalg.pinv(cov1)
icov2 = np.linalg.pinv(cov2)

In [7]:
import math

(sign1,logdet1) = np.linalg.slogdet(cov1)
(sign2,logdet2) = np.linalg.slogdet(cov2)
det1 = sign1 * np.exp(logdet1)
det2 = sign2 * np.exp(logdet2)
 
labels = np.zeros((len(train_labels),1))

In [8]:
for i in range(0, len(train_labels)):
    train_mu1 = (train_data[i,:] - mu1).reshape(1,20)   
    train_mu2 = (train_data[i,:] - mu2).reshape(1,20)
    
    likelihood1 = (1/(2*math.pi*math.sqrt(det1)))*np.dot(np.dot(-0.5*train_mu1, icov1),train_mu1.reshape(20,1))
    likelihood2 = (1/(2*math.pi*math.sqrt(det2)))*np.dot(np.dot(-0.5*train_mu2, icov2),train_mu2.reshape(20,1))
    #print("Likelihood1:", likelihood1, "Likelihood2: ", likelihood2)
    if np.abs(likelihood1 * sd1) > np.abs(likelihood2 * sd2):
       labels[i] = 1
    else:
       labels[i] = 0


In [9]:
count_0 = 0
count_1 = 0
for i in range(0,len(train_labels)):
    #if y_test[i] < 0.4:
     if  (labels[i] == 0):
       count_0 += 1
     else:
       count_1 += 1
print("predicted number of female", count_0)
print("predicted number of male", count_1)
print("total number of train data:",count_0 + count_1)

predicted number of female 2454
predicted number of male 80
total number of train data: 2534


In [10]:
correct_predic = 0
wrong_predic = 0
for i in range(0,len(train_labels)):
    if labels[i] == train_labels[i]:
       correct_predic += 1
    else:
       wrong_predic += 1
    
print("Number of correct prediction", correct_predic)
print("Number of wrong prediction", wrong_predic)

Number of correct prediction 1299
Number of wrong prediction 1235


In [11]:
print("Correct rate is: ", correct_predic/(wrong_predic + correct_predic))

Correct rate is:  0.5126282557221784
